In [2]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model

import numpy as np
import cv2
import os

In [3]:
prototxtPath = os.path.sep.join([r'F:\UIU\Semester 11\Pattern Lab\Project_mask\face_detector','deploy.prototxt'])
weightsPath = os.path.sep.join([r'F:\UIU\Semester 11\Pattern Lab\Project_mask\face_detector','res10_300x300_ssd_iter_140000.caffemodel'])

In [4]:
prototxtPath

'F:\\UIU\\Semester 11\\Pattern Lab\\Project_mask\\face_detector\\deploy.prototxt'

In [5]:
weightsPath

'F:\\UIU\\Semester 11\\Pattern Lab\\Project_mask\\face_detector\\res10_300x300_ssd_iter_140000.caffemodel'

In [6]:
net = cv2.dnn.readNet(prototxtPath,weightsPath)

In [7]:
model = load_model(r'F:\UIU\Semester 11\Pattern Lab\Project_mask\mobilenet_v2.model')

In [18]:
image = cv2.imread(r'F:\UIU\Semester 11\Pattern Lab\Project_mask\examples\mask3.jpeg')

In [19]:
image

array([[[204, 212, 212],
        [204, 212, 212],
        [205, 213, 212],
        ...,
        [230, 234, 209],
        [226, 232, 207],
        [226, 232, 207]],

       [[204, 212, 212],
        [205, 213, 213],
        [206, 214, 213],
        ...,
        [230, 234, 209],
        [226, 232, 207],
        [226, 232, 207]],

       [[205, 213, 213],
        [206, 214, 214],
        [207, 215, 214],
        ...,
        [230, 235, 208],
        [227, 233, 208],
        [227, 233, 208]],

       ...,

       [[ 18,  18,  18],
        [ 19,  19,  19],
        [ 22,  22,  22],
        ...,
        [ 89,  75,  63],
        [ 88,  75,  67],
        [ 88,  77,  69]],

       [[ 22,  22,  22],
        [ 25,  25,  25],
        [ 29,  29,  29],
        ...,
        [ 88,  76,  64],
        [ 86,  77,  68],
        [ 87,  77,  70]],

       [[ 26,  26,  26],
        [ 29,  29,  29],
        [ 34,  34,  34],
        ...,
        [ 87,  77,  67],
        [ 87,  78,  69],
        [ 87,  79,  72]]

In [20]:
h , w= image.shape[:2]

In [21]:
(h,w)

(1280, 960)

In [22]:
blob =cv2.dnn.blobFromImage(image,1.0,(300,300),(104.0,177.0,123.0))

In [23]:
blob

array([[[[ 102.,  103.,  106., ...,  151.,  140.,  123.],
         [ 106.,  108.,  109., ...,  141.,  139.,  129.],
         [ 107.,  110.,  110., ...,  150.,  137.,  133.],
         ...,
         [ -89.,  -89.,  -91., ...,    1.,    3.,    2.],
         [ -88.,  -88.,  -90., ...,    4.,    4.,   -4.],
         [ -83.,  -70.,  -64., ...,  -11.,   -9.,  -17.]],

        [[  37.,   38.,   41., ...,   78.,   69.,   56.],
         [  41.,   43.,   44., ...,   68.,   64.,   59.],
         [  42.,   45.,   44., ...,   76.,   62.,   62.],
         ...,
         [-162., -162., -162., ..., -101.,  -96.,  -93.],
         [-161., -161., -161., ...,  -99.,  -96.,  -97.],
         [-156., -143., -135., ..., -103., -100., -102.]],

        [[  91.,   90.,   89., ...,   94.,   93.,   84.],
         [  95.,   95.,   92., ...,   84.,   86.,   87.],
         [  96.,   97.,   92., ...,  103.,   85.,   94.],
         ...,
         [-108., -108., -108., ...,  -74.,  -68.,  -57.],
         [-107., -107., -1

In [24]:
blob.shape

(1, 3, 300, 300)

In [25]:
net.setInput(blob)
detections = net.forward()

In [26]:
detections.shape[2]


200

In [ ]:
#loop over the detections

for i in range (0,detections.shape[2]):
    
    confidence = detections[0,0,i,2]
    if confidence>0.5:
        
        #we need the X,Y co ordinates
        box = detections[0,0,i,3:7]*np.array([w,h,w,h])
        (startX,startY,endX,endY) = box.astype('int')
        
        #ensure the bounding boxes fall within the dimension of the frame
        
        (startX,startY) = (max(0,startX),max(0,startY))
        (endX,endY) = (min(w-1,endX),min(h-1,endY))
        
        #extract the face ROI , convert it from BGR to RGB channel, resize it to 224,224, preprocess it
        
        face = image[startY:endY, startX:endX]
        print(face)
        
        face = cv2.cvtColor(face,cv2.COLOR_BGR2RGB)
        
        face = cv2.resize(face,(224,224))
        
        face = img_to_array(face)
        face = preprocess_input(face)
        face = np.expand_dims(face,axis = 0)
        
        (mask,withoutMask) = model.predict(face)[0]
        
        #determinde the class label and color we will use to draw the bounding box and text
        
        label = 'Mask' if mask>withoutMask else 'No Mask'
        
        color = (0,255,0) if label=='Mask' else (0,0,255)
        
        #include the probability in the label
        
        label = "{}:{:2f}%".format(label,max(mask,withoutMask)*100)
        
        #display the label and bounding boxes
        
        cv2.putText(image,label,(startX,startY-10),cv2.FONT_HERSHEY_SIMPLEX,0.45,color,2)
        cv2.rectangle(image,(startX,startY),(endX,endY),color,2)
        

cv2.imshow("OutPut",image)
cv2.waitKey(0)
cv2.destroyAllwindows()

[[[114  99 143]
  [105  95 125]
  [ 94  87 108]
  ...
  [140 142 142]
  [150 151 149]
  [153 153 147]]

 [[108  95 133]
  [102  92 115]
  [ 93  85 102]
  ...
  [120 123 127]
  [126 131 130]
  [130 133 131]]

 [[106  94 124]
  [103  93 109]
  [ 99  90 103]
  ...
  [125 127 137]
  [126 130 135]
  [132 137 140]]

 ...

 [[ 87  46  31]
  [ 88  48  30]
  [ 88  48  30]
  ...
  [ 33  33  45]
  [ 39  41  52]
  [ 34  38  49]]

 [[ 87  46  31]
  [ 88  48  30]
  [ 88  48  30]
  ...
  [ 32  34  45]
  [ 32  36  47]
  [ 42  45  59]]

 [[ 87  46  31]
  [ 88  48  30]
  [ 88  48  30]
  ...
  [ 32  36  47]
  [ 27  33  44]
  [ 50  56  69]]]
